In [1]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import metrics   

from sklearn import set_config
set_config(display='diagram') # Useful for display the pipeline

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
#dataset_half_S = pd.read_csv('Datasets/dataset_halfSecondWindow.csv', index_col=[0])
dataset = pd.read_csv('Datasets/dataset_5SecondWindow.csv')

In [4]:
dataset.head(10)

,Unnamed: 0,id,time,activityrecognition#0,activityrecognition#1,android.sensor.accelerometer#mean,android.sensor.accelerometer#min,android.sensor.accelerometer#max,android.sensor.accelerometer#std,android.sensor.game_rotation_vector#mean,android.sensor.game_rotation_vector#min,android.sensor.game_rotation_vector#max,android.sensor.game_rotation_vector#std,android.sensor.gravity#mean,android.sensor.gravity#min,android.sensor.gravity#max,android.sensor.gravity#std,android.sensor.gyroscope#mean,android.sensor.gyroscope#min,android.sensor.gyroscope#max,android.sensor.gyroscope#std,android.sensor.gyroscope_uncalibrated#mean,android.sensor.gyroscope_uncalibrated#min,android.sensor.gyroscope_uncalibrated#max,android.sensor.gyroscope_uncalibrated#std,android.sensor.light#mean,android.sensor.light#min,android.sensor.light#max,android.sensor.light#std,android.sensor.linear_acceleration#mean,android.sensor.linear_acceleration#min,android.sensor.linear_acceleration#max,android.sensor.linear_acceleration#std,android.sensor.magnetic_field#mean,android.sensor.magnetic_field#min,android.sensor.magnetic_field#max,android.sensor.magnetic_field#std,android.sensor.magnetic_field_uncalibrated#mean,android.sensor.magnetic_field_uncalibrated#min,android.sensor.magnetic_field_uncalibrated#max,android.sensor.magnetic_field_uncalibrated#std,android.sensor.orientation#mean,android.sensor.orientation#min,android.sensor.orientation#max,android.sensor.orientation#std,android.sensor.pressure#mean,android.sensor.pressure#min,android.sensor.pressure#max,android.sensor.pressure#std,android.sensor.proximity#mean,android.sensor.proximity#min,android.sensor.proximity#max,android.sensor.proximity#std,android.sensor.rotation_vector#mean,android.sensor.rotation_vector#min,android.sensor.rotation_vector#max,android.sensor.rotation_vector#std,android.sensor.step_counter#mean,android.sensor.step_counter#min,android.sensor.step_counter#max,android.sensor.step_counter#std,sound#mean,sound#min,sound#max,sound#std,speed#mean,speed#min,speed#max,speed#std,target,user
0,0,16170,78.0,NaN,100.0,9.811476,9.758895,9.849411,0.014626,0.029340,0.029014,0.029526,0.000119,9.806650,9.806649,9.806651,4.780692e-07,0.001651,0.000000,0.003533,0.000737,0.016221,0.014172,0.018695,0.000982,0.000000,0.0,0.0,0.000000,0.020978,0.002495,0.052410,0.011045,57.099638,56.690387,57.575950,0.177549,51.363566,51.199707,51.539208,0.080899,354.286933,353.598335,354.942707,0.245676,1004.090261,1004.05540,1004.12790,0.017416,8.000000,8.0,8.0,NaN,0.050413,0.044777,0.056351,0.002109,28966.00,28966.0,28966.0,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,U12
1,1,15871,145.0,NaN,100.0,9.939207,7.707437,17.146631,1.775944,0.999925,0.999903,0.999946,0.000030,9.806624,9.806624,9.806624,6.474977e-07,0.036326,0.011669,0.059388,0.020290,0.039023,0.014132,0.085494,0.018629,0.000000,0.0,0.0,NaN,0.879220,0.641117,1.185810,0.278730,29.351288,28.172505,30.386017,0.921547,82.767760,82.409890,83.125630,0.506105,332.695577,330.461054,339.108607,1.705816,1008.274660,1008.27466,1008.27466,NaN,NaN,NaN,NaN,NaN,0.999981,0.999963,0.999999,0.000026,NaN,NaN,NaN,NaN,89.200210,89.065143,89.335277,0.191013,16.539349,16.539349,16.539349,0.628595,Car,U12
2,2,16811,150.0,NaN,100.0,9.827178,9.804817,9.849262,0.011199,0.665215,0.665213,0.665218,0.000004,9.806650,9.806650,9.806650,2.821568e-07,0.001525,0.000000,0.002609,0.000713,0.016302,0.014645,0.017374,0.000876,2.000000,2.0,2.0,0.707107,0.034110,0.025709,0.042511,0.011881,29.904927,29.741722,30.038309,0.130851,55.695716,55.501802,55.868506,0.128945,74.752653,73.223493,75.268574,0.413704,1005.977540,1005.97754,1005.97754,NaN,8.000000,8.0,8.0,NaN,0.610456,0.610456,0.610456,0.011312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,U12
3,3,15831,105.0,NaN,77.0,9.673039,7.659674,12.304298,0.862553,0.996221,0.993781,0.998661,0.003451,9.806638,9.806638,9.806638,2.120564e-06,0.036440,0.020463,0.052512,0.010553,0.050759,0.024732,0.081887,0.018037,0.000000,0

In [5]:
dataset.user.unique().tolist() #all users

['U12',
 'U1',
 'U8',
 'U7',
 'U10',
 'U13',
 'U6',
 'U11',
 'U3',
 'U2',
 'U4',
 'U9',
 'U5']

In [6]:
for name, group in dataset.groupby('user'):
    print(name,len(group)) 

U1 2470
U10 531
U11 167
U12 820
U13 90
U2 271
U3 331
U4 227
U5 17
U6 279
U7 397
U8 214
U9 79


In [7]:
dataset.target.unique().tolist() #all users

['Still', 'Car', 'Train', 'Bus', 'Walking']

In [8]:
dataset.target = dataset.target.replace({'Car':'Vehicle','Bus':'Vehicle'})

In [9]:
for name, group in dataset.groupby('target'):
    print(name,len(group)) 

Still 1179
Train 1179
Vehicle 2358
Walking 1177


In [10]:
# encode the target
#dataset.target = dataset.target.replace({'Still':0, 'Walking':1, 'Car':2, 'Bus':3, 'Train':4})

In [11]:
#dataset.columns

id
time
activityrecognition # 2
android.sensor.accelerometer #mmms //Measures the acceleration force in m/s2 that is applied to a device on all three physical axes (x, y, and z), including the force of gravityacceleration force
android.sensor.game_rotation_vector # mmms //orientation of the device
android.sensor.gravity #mmms //Measures the force of gravity in m/s2 that is applied to a device on all three physical axes (x, y, z)
android.sensor.gyroscope #mmms //Measures a device's rate of rotation in rad/s around each of the three physical axes (x, y, and z)
android.sensor.light #mmms //Measures the ambient light level (illumination) in lx
android.sensor.linear_acceleration #mmms // Measures the acceleration force in m/s2 that is applied to a device on all three physical axes (x, y, and z), excluding the force of gravity
android.sensor.magnetic_field #mmms //Measures the ambient geomagnetic field for all three physical axes (x, y, z) in μT
android.sensor.magnetic_field_uncalibrated #mmms
android.sensor.orientation #mmms
android.sensor.pressure #mmms //Measures the ambient air pressure in hPa or mbar.
android.sensor.proximity #mmms //Measures the proximity of an object in cm relative to the view screen of a device. This sensor is typically used to determine whether a handset is being held up to a person's ear.
android.sensor.rotation_vector #mmms //Measures the orientation of a device by providing the three elements of the device's rotation vector.
android.sensor.step_counter #mmms
sound #mmms
speed #mmms
target
user

targets : walking, car, still, train and bus. 
26% of data is annoted as walking, 25% as driving a car, 24% as standing still, 20% as being on train, and 5% as being on bus. 

In [12]:
#dataset.info()

In [13]:
dataset.describe()

,Unnamed: 0,id,time,activityrecognition#0,activityrecognition#1,android.sensor.accelerometer#mean,android.sensor.accelerometer#min,android.sensor.accelerometer#max,android.sensor.accelerometer#std,android.sensor.game_rotation_vector#mean,android.sensor.game_rotation_vector#min,android.sensor.game_rotation_vector#max,android.sensor.game_rotation_vector#std,android.sensor.gravity#mean,android.sensor.gravity#min,android.sensor.gravity#max,android.sensor.gravity#std,android.sensor.gyroscope#mean,android.sensor.gyroscope#min,android.sensor.gyroscope#max,android.sensor.gyroscope#std,android.sensor.gyroscope_uncalibrated#mean,android.sensor.gyroscope_uncalibrated#min,android.sensor.gyroscope_uncalibrated#max,android.sensor.gyroscope_uncalibrated#std,android.sensor.light#mean,android.sensor.light#min,android.sensor.light#max,android.sensor.light#std,android.sensor.linear_acceleration#mean,android.sensor.linear_acceleration#min,android.sensor.linear_acceleration#max,android.sensor.linear_acceleration#std,android.sensor.magnetic_field#mean,android.sensor.magnetic_field#min,android.sensor.magnetic_field#max,android.sensor.magnetic_field#std,android.sensor.magnetic_field_uncalibrated#mean,android.sensor.magnetic_field_uncalibrated#min,android.sensor.magnetic_field_uncalibrated#max,android.sensor.magnetic_field_uncalibrated#std,android.sensor.orientation#mean,android.sensor.orientation#min,android.sensor.orientation#max,android.sensor.orientation#std,android.sensor.pressure#mean,android.sensor.pressure#min,android.sensor.pressure#max,android.sensor.pressure#std,android.sensor.proximity#mean,android.sensor.proximity#min,android.sensor.proximity#max,android.sensor.proximity#std,android.sensor.rotation_vector#mean,android.sensor.rotation_vector#min,android.sensor.rotation_vector#max,android.sensor.rotation_vector#std,android.sensor.step_counter#mean,android.sensor.step_counter#min,android.sensor.step_counter#max,android.sensor.step_counter#std,sound#mean,sound#min,sound#max,sound#std,speed#mean,speed#min,speed#max,speed#std
count,5893.0000,5893.000000,5893.000000,0.0,5893.000000,5893.000000,5893.000000,5893.000000,5893.000000,5094.000000,5094.000000,5094.000000,5093.000000,5396.000000,5396.000000,5396.000000,5.356000e+03,5173.000000,5173.000000,5173.000000,5172.000000,5094.000000,5094.000000,5094.000000,5094.000000,4381.000000,4381.000000,4381.000000,2124.000000,5399.000000,5399.000000,5399.000000,5370.000000,5400.000000,5400.000000,5400.000000,5383.000000,5094.000000,5094.000000,5094.000000,5087.000000,5396.000000,5396.000000,5396.000000,5373.000000,1918.000000,1918.000000,1918.000000,1128.000000,1119.000000,1119.000000,1119.000000,283.000000,5172.000000,5172.000000,5172.000000,5163.000000,1487.000000,1487.000000,1487.000000,232.000000,4899.000000,4899.000000,4899.000000,4062.000000,3520.000000,3520.000000,3520.000000,1318.000000
mean,2946.0000,10685.407263,117.731716,NaN,87.718310,10.007976,8.426262,12.247691,0.982987,0.759280,0.743872,0.771629,0.011876,9.806561,9.806530,9.806592,1.220395e-04,0.312536,0.089494,0.735042,0.189903,0.312807,0.125427,0.614961,0.174279,288.825545,254.140861,331.698065,85.357327,1.315971,0.863086,2.157592,0.602771,56.212508,54.267805,58.207109,1.772942,138.409066,136.955896,139.907248,2.038359,212.629637,199.930476,224.731494,12.488254,1016.203139,1016.189143,1016.218816,0.023097,6.117128,5.174263,6.797140,4.081700,0.765932,0.750273,0.779000,0.012977,10112.523523,10112.205783,10112.769334,2.959475,78.724855,78.451658,78.997213,1.913065,8.235254,8.216747,8.253528,0.236959
std,1701.3069,6921.229833,126.474151,NaN,24.562728,0.655949,1.783007,5.178436,1.505615,0.251174,0.255232,0.251401,0.026647,0.001610,0.001709,0.001584,7.681037e-04,0.554585,0.206327,1.356240,0.342901,0.537925,0.235223,1.112451,0.322397,1761.573894,1361.427348,2379.942459,1052.057842,2.065444,1.493150,5.058223,1.136752,38.210015,37.902702,38.681821,2.938644,71.034674,70.764992,71.910605,5.423627,95.253318,98.160410,101.113269,31.517931,14.142150,14.

In [14]:
dataset.isnull().sum()

Unnamed: 0                                            0
id                                                    0
time                                                  0
activityrecognition#0                              5893
activityrecognition#1                                 0
android.sensor.accelerometer#mean                     0
android.sensor.accelerometer#min                      0
android.sensor.accelerometer#max                      0
android.sensor.accelerometer#std                      0
android.sensor.game_rotation_vector#mean            799
android.sensor.game_rotation_vector#min             799
android.sensor.game_rotation_vector#max             799
android.sensor.game_rotation_vector#std             800
android.sensor.gravity#mean                         497
android.sensor.gravity#min                          497
android.sensor.gravity#max                          497
android.sensor.gravity#std                          537
android.sensor.gyroscope#mean                   

In [15]:
# replace anroid.sensor & #
dataset.columns = dataset.columns.str.replace('android.sensor.','').str.replace('#','_')
dataset.head()

<ipython-input-15-e92dca201964>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  dataset.columns = dataset.columns.str.replace('android.sensor.','').str.replace('#','_')


,Unnamed: 0,id,time,activityrecognition_0,activityrecognition_1,accelerometer_mean,accelerometer_min,accelerometer_max,accelerometer_std,game_rotation_vector_mean,game_rotation_vector_min,game_rotation_vector_max,game_rotation_vector_std,gravity_mean,gravity_min,gravity_max,gravity_std,gyroscope_mean,gyroscope_min,gyroscope_max,gyroscope_std,gyroscope_uncalibrated_mean,gyroscope_uncalibrated_min,gyroscope_uncalibrated_max,gyroscope_uncalibrated_std,light_mean,light_min,light_max,light_std,linear_acceleration_mean,linear_acceleration_min,linear_acceleration_max,linear_acceleration_std,magnetic_field_mean,magnetic_field_min,magnetic_field_max,magnetic_field_std,magnetic_field_uncalibrated_mean,magnetic_field_uncalibrated_min,magnetic_field_uncalibrated_max,magnetic_field_uncalibrated_std,orientation_mean,orientation_min,orientation_max,orientation_std,pressure_mean,pressure_min,pressure_max,pressure_std,proximity_mean,proximity_min,proximity_max,proximity_std,rotation_vector_mean,rotation_vector_min,rotation_vector_max,rotation_vector_std,step_counter_mean,step_counter_min,step_counter_max,step_counter_std,sound_mean,sound_min,sound_max,sound_std,speed_mean,speed_min,speed_max,speed_std,target,user
0,0,16170,78.0,NaN,100.0,9.811476,9.758895,9.849411,0.014626,0.029340,0.029014,0.029526,0.000119,9.806650,9.806649,9.806651,4.780692e-07,0.001651,0.000000,0.003533,0.000737,0.016221,0.014172,0.018695,0.000982,0.0,0.0,0.0,0.000000,0.020978,0.002495,0.052410,0.011045,57.099638,56.690387,57.575950,0.177549,51.363566,51.199707,51.539208,0.080899,354.286933,353.598335,354.942707,0.245676,1004.090261,1004.05540,1004.12790,0.017416,8.0,8.0,8.0,NaN,0.050413,0.044777,0.056351,0.002109,28966.0,28966.0,28966.0,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,U12
1,1,15871,145.0,NaN,100.0,9.939207,7.707437,17.146631,1.775944,0.999925,0.999903,0.999946,0.000030,9.806624,9.806624,9.806624,6.474977e-07,0.036326,0.011669,0.059388,0.020290,0.039023,0.014132,0.085494,0.018629,0.0,0.0,0.0,NaN,0.879220,0.641117,1.185810,0.278730,29.351288,28.172505,30.386017,0.921547,82.767760,82.409890,83.125630,0.506105,332.695577,330.461054,339.108607,1.705816,1008.274660,1008.27466,1008.27466,NaN,NaN,NaN,NaN,NaN,0.999981,0.999963,0.999999,0.000026,NaN,NaN,NaN,NaN,89.200210,89.065143,89.335277,0.191013,16.539349,16.539349,16.539349,0.628595,Vehicle,U12
2,2,16811,150.0,NaN,100.0,9.827178,9.804817,9.849262,0.011199,0.665215,0.665213,0.665218,0.000004,9.806650,9.806650,9.806650,2.821568e-07,0.001525,0.000000,0.002609,0.000713,0.016302,0.014645,0.017374,0.000876,2.0,2.0,2.0,0.707107,0.034110,0.025709,0.042511,0.011881,29.904927,29.741722,30.038309,0.130851,55.695716,55.501802,55.868506,0.128945,74.752653,73.223493,75.268574,0.413704,1005.977540,1005.97754,1005.97754,NaN,8.0,8.0,8.0,NaN,0.610456,0.610456,0.610456,0.011312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,Still,U12
3,3,15831,105.0,NaN,77.0,9.673039,7.659674,12.304298,0.862553,0.996221,0.993781,0.998661,0.003451,9.806638,9.806638,9.806638,2.120564e-06,0.036440,0.020463,0.052512,0.010553,0.050759,0.024732,0.081887,0.018037,0.0,0.0,0.0,NaN,1.488361,0.575910,2.415879,0.920077,37.836471,37.525725,38.147217,0.439461,95.743821,95.664309,95.823332,0.112446,373.556039,359.897250,397.071741,8.870011,1006.950440,1006.95044,1006.95044,NaN,NaN,NaN,NaN,NaN,0.998112,0.998112,0.998112,0.002788,NaN,NaN,NaN,NaN,87.470377,87.470377,87.470377,2.284186,17.739895,17.739895,17.739895,0.628595,Vehicle,U12
4,4,876,77.0,NaN,100.0,9.993466,8.965621,10.891645,0.504117,0.563792,0.521799,0.672664,0.064792,9.806650,9.806650,9.806651,2.644430e-07,0.183202,0.020667,0.380875,0.098819,0.265652,0.141208,0.313020,0.083175,30.0,30.0,30.0,39.597980,1.658308,0.379803,2.795995,1.122225,78.375718,76.543261,79.273739,1.104361,157.429218,156.795909,158.062526,0.895633,193.201458,35.864972,360.560468,181.729455,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.322242,0.308055,0.378193,0.025608,NaN,NaN,NaN,NaN,89.770732,89.770732,89.7707

In [16]:
dataset['rv_gyro_mean'] = dataset['rotation_vector_mean'] * dataset['gyroscope_mean']
dataset['lin_speed_mean'] = dataset['linear_acceleration_mean'] * dataset['speed_mean']
dataset['rv_gyro_prox_mean'] = dataset['rotation_vector_mean'] * dataset['gyroscope_mean']* dataset['proximity_std']
dataset['lin_speed_prox_mean'] = dataset['linear_acceleration_mean'] * dataset['speed_mean'] *dataset['proximity_std']
dataset['rv_gyro__grv_mean'] = dataset['rotation_vector_mean'] * dataset['gyroscope_mean']* dataset['game_rotation_vector_mean']

In [17]:
dataset = dataset[(['accelerometer_min',
                    'accelerometer_max',
                    'accelerometer_std',
                    'gyroscope_mean',
                    'gyroscope_uncalibrated_mean',
                    'gyroscope_std',
                    'linear_acceleration_mean',
                    'rotation_vector_mean',
                    'rotation_vector_max',
                    'proximity_std',
                    'magnetic_field_uncalibrated_min',
                    'sound_min',
                    'sound_max',
                    'sound_mean',
                    
                    'rv_gyro_mean',
                    
                    'lin_speed_mean',
                    'rv_gyro_prox_mean',
                    'lin_speed_prox_mean',
                    'rv_gyro__grv_mean',
                    
                    'user',
                    'target'
                 ])]

In [18]:
dataset.columns

Index(['accelerometer_min', 'accelerometer_max', 'accelerometer_std',
       'gyroscope_mean', 'gyroscope_uncalibrated_mean', 'gyroscope_std',
       'linear_acceleration_mean', 'rotation_vector_mean',
       'rotation_vector_max', 'proximity_std',
       'magnetic_field_uncalibrated_min', 'sound_min', 'sound_max',
       'sound_mean', 'rv_gyro_mean', 'lin_speed_mean', 'rv_gyro_prox_mean',
       'lin_speed_prox_mean', 'rv_gyro__grv_mean', 'user', 'target'],
      dtype='object')

In [19]:
dataset.head()

,accelerometer_min,accelerometer_max,accelerometer_std,gyroscope_mean,gyroscope_uncalibrated_mean,gyroscope_std,linear_acceleration_mean,rotation_vector_mean,rotation_vector_max,proximity_std,magnetic_field_uncalibrated_min,sound_min,sound_max,sound_mean,rv_gyro_mean,lin_speed_mean,rv_gyro_prox_mean,lin_speed_prox_mean,rv_gyro__grv_mean,user,target
0,9.758895,9.849411,0.014626,0.001651,0.016221,0.000737,0.020978,0.050413,0.056351,NaN,51.199707,NaN,NaN,NaN,0.000083,0.000000,NaN,NaN,0.000002,U12,Still
1,7.707437,17.146631,1.775944,0.036326,0.039023,0.020290,0.879220,0.999981,0.999999,NaN,82.409890,89.065143,89.335277,89.200210,0.036325,14.541729,NaN,NaN,0.036322,U12,Vehicle
2,9.804817,9.849262,0.011199,0.001525,0.016302,0.000713,0.034110,0.610456,0.610456,NaN,55.501802,NaN,NaN,NaN,0.000931,0.000000,NaN,NaN,0.000619,U12,Still
3,7.659674,12.304298,0.862553,0.036440,0.050759,0.010553,1.488361,0.998112,0.998112,NaN,95.664309,87.470377,87.470377,87.470377,0.036372,26.403360,NaN,NaN,0.036234,U12,Vehicle
4,8.965621,10.891645,0.504117,0.183202,0.265652,0.098819,1.658308,0.322242,0.378193,NaN,156.795909,89.770732,89.770732,89.770732,0.059035,14.924768,NaN,NaN,0.033284,U1,Vehicle


In [20]:
# split user = ['Luca' 'Pierpaolo' 'IvanHeibi']
dataset_test = dataset[(dataset['user'] == 'U2')|(dataset['user'] == 'U3')|(dataset['user'] == 'U4')]
dataset_train = dataset[(dataset['user'] != 'U2')&(dataset['user'] != 'U3')&(dataset['user'] != 'U4')]
print(len(dataset))
print(len(dataset_test))
print(len(dataset_train))

5893
829
5064


In [21]:
dataset_train.sample(10)

,accelerometer_min,accelerometer_max,accelerometer_std,gyroscope_mean,gyroscope_uncalibrated_mean,gyroscope_std,linear_acceleration_mean,rotation_vector_mean,rotation_vector_max,proximity_std,magnetic_field_uncalibrated_min,sound_min,sound_max,sound_mean,rv_gyro_mean,lin_speed_mean,rv_gyro_prox_mean,lin_speed_prox_mean,rv_gyro__grv_mean,user,target
5605,7.387235,14.785897,2.909702,1.238141,1.083877,0.701909,5.520587,0.968398,0.993752,NaN,122.183364,89.811810,89.811810,89.811810,1.199013,0.000000,NaN,NaN,0.965302,U1,Walking
649,6.768631,10.869313,1.025717,0.075487,0.079328,0.050744,0.271361,0.665622,0.668664,NaN,77.113554,NaN,NaN,NaN,0.050246,5.332316,NaN,NaN,0.047147,U11,Vehicle
1148,9.467043,9.818085,0.100071,0.165925,0.139364,0.167080,0.626307,0.981925,0.984376,NaN,140.788759,89.741353,89.741353,89.741353,0.162926,0.000000,NaN,NaN,0.132168,U1,Vehicle
3933,8.720439,10.650946,0.235390,0.026146,0.102027,0.170897,0.020173,0.635084,0.647548,NaN,274.829891,60.788282,60.788282,60.788282,0.016605,0.000000,NaN,NaN,0.007586,U10,Still
45,9.314642,10.624079,0.404480,0.060411,0.040011,0.039301,0.491550,0.639666,0.639666,NaN,79.168807,NaN,NaN,NaN,0.038643,5.755992,NaN,NaN,0.022029,U11,Vehicle
4736,5.502987,21.625198,4.980574,1.608670,1.299858,0.816322,4.878485,0.889258,0.927132,NaN,131.542619,89.742202,89.742202,89.742202,1.430523,NaN,NaN,NaN,1.247695,U1,Walking
2996,9.375310,10.778821,0.380759,0.024610,0.024469,0.016097,0.373871,0.938889,0.940308,NaN,527.032479,78.909372,78.909372,78.909372,0.023106,6.253573,NaN,NaN,0.014062,U13,Vehicle
5785,9.923408,9.971747,0.016067,0.007795,0.039998,0.004523,0.020494,0.998706,0.998706,NaN,141.790000,87.041334,87.041334,87.041334,0.007785,NaN,NaN,NaN,0.007379,U1,Still
596,9.796124,9.901972,0.023459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.232394,48.232394,48.232394,NaN,NaN,NaN,NaN,NaN,U8,Still
5072,8.981395,10.628983,0.254588,0.107043,0.073171,0.122092,0.990220,0.727186,0.728891,NaN,244.434368,76.491628,76.491628,76.491628,0.077840,0.000000,NaN,NaN,0.035829,U10,Still


In [22]:
#dataset_train = dataset_train.fillna(0)

In [23]:
dataset_test.head()

,accelerometer_min,accelerometer_max,accelerometer_std,gyroscope_mean,gyroscope_uncalibrated_mean,gyroscope_std,linear_acceleration_mean,rotation_vector_mean,rotation_vector_max,proximity_std,magnetic_field_uncalibrated_min,sound_min,sound_max,sound_mean,rv_gyro_mean,lin_speed_mean,rv_gyro_prox_mean,lin_speed_prox_mean,rv_gyro__grv_mean,user,target
33,9.764513,14.028126,1.283204,0.615399,0.778458,0.655962,1.721314,0.955202,0.955202,NaN,36.378866,77.064227,77.064227,77.064227,0.587830,2.050172,NaN,NaN,0.554815,U3,Walking
49,3.324387,14.608536,4.031835,0.597423,0.619693,0.167046,4.223926,0.934642,0.937574,NaN,87.072128,77.221125,77.221125,77.221125,0.558377,NaN,NaN,NaN,0.555901,U2,Walking
51,4.595131,17.636696,3.551271,NaN,NaN,NaN,2.791507,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U4,Walking
54,8.712177,9.963264,0.361961,0.447802,0.334574,0.116564,0.958200,0.983561,0.989576,NaN,77.496043,74.178611,74.178611,74.178611,0.440440,0.000000,NaN,NaN,0.393798,U3,Vehicle
56,5.371977,15.088214,3.438894,0.550989,0.749501,0.250989,3.610910,0.802652,0.823187,NaN,72.877824,78.588379,80.072657,79.330518,0.442252,4.333092,NaN,NaN,0.441829,U2,Walking


In [24]:
# drop user in test & split
dataset_train = dataset_train.drop(['user'], axis = 1)
dataset_test = dataset_test.drop(['user'], axis = 1)

In [25]:
num_vars = ['accelerometer_min',
            'accelerometer_max',
            'accelerometer_std',
            'gyroscope_mean',
            'gyroscope_uncalibrated_mean',
            'gyroscope_std',
            
            'linear_acceleration_mean',
            
            'rotation_vector_mean',
            'rotation_vector_max',
            'proximity_std',
            'magnetic_field_uncalibrated_min',
            'sound_min',
            'sound_max',
            'sound_mean',
            
            
            'rv_gyro_mean',
            'lin_speed_mean',
            'rv_gyro_prox_mean',
            'lin_speed_prox_mean',
            'rv_gyro__grv_mean',
            ]


In [26]:
# YOUR CODE HERE
from sklearn.impute import KNNImputer
from sklearn.preprocessing import RobustScaler
num_4_treeModels = pipeline.Pipeline(steps=[
  ('imputer', impute.KNNImputer(n_neighbors=3)),
  ('Normalizer',RobustScaler())
  
])

tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_4_treeModels, num_vars)
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

tree_prepro

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  KNNImputer(n_neighbors=3)),
                                                 ('Normalizer',
                                                  RobustScaler())]),
                                 ['accelerometer_min', 'accelerometer_max',
                                  'accelerometer_std', 'gyroscope_mean',
                                  'gyroscope_uncalibrated_mean',
                                  'gyroscope_std', 'linear_acceleration_mean',
                                  'rotation_vector_mean', 'rotation_vector_max',
                                  'proximity_std',
                                  'magnetic_field_uncalibrated_min',
                                  'sound_min', 'sound_max', 'sound_mean',
                                  'rv_gyro_mean', 'lin_speed_mean',
                                  'rv_gyro_prox_mean', 'lin_speed_prox_mean',
                                  'rv_gyro__grv_mean'])])

In [27]:


from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from catboost              import CatBoostClassifier



In [28]:
tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees":ExtraTreesClassifier(),
  "Random Forest":RandomForestClassifier(),
  "AdaBoost":AdaBoostClassifier(),
  "Skl GBM":GradientBoostingClassifier(),
  "Skl HistGBM":GradientBoostingClassifier(),
  "XGBoost":XGBClassifier(),
  "LightGBM":LGBMClassifier(),
  "CatBoost":CatBoostClassifier()
}

tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}



tree_classifiers["LightGBM"]

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(n_neighbors=3)),
                                                                  ('Normalizer',
                                                                   RobustScaler())]),
                                                  ['accelerometer_min',
                                                   'accelerometer_max',
                                                   'accelerometer_std',
                                                   'gyroscope_mean',
                                                   'gyroscope_uncalibrated_mean',
                                                   'gyroscope_std',
                                                   'linear_acceleration_mean',
                                                   'rotation_vector_mean',
                                                   'rotation_vector_max',
                                                   'proximity_std',
                                                   'magnetic_field_uncalibrated_min',
                                                   'sound_min', 'sound_max',
                                                   'sound_mean', 'rv_gyro_mean',
                                                   'lin_speed_mean',
                                                   'rv_gyro_prox_mean',
                                                   'lin_speed_prox_mean',
                                                   'rv_gyro__grv_mean'])])),
                ('lgbmclassifier', LGBMClassifier())])

In [29]:
# split 
x_train = dataset_train.drop(['target'], axis=1)
y_train = dataset_train.target
x_test = dataset_test.drop(['target'], axis=1)
y_test = dataset_test.target

In [30]:
x_train.columns

Index(['accelerometer_min', 'accelerometer_max', 'accelerometer_std',
       'gyroscope_mean', 'gyroscope_uncalibrated_mean', 'gyroscope_std',
       'linear_acceleration_mean', 'rotation_vector_mean',
       'rotation_vector_max', 'proximity_std',
       'magnetic_field_uncalibrated_min', 'sound_min', 'sound_max',
       'sound_mean', 'rv_gyro_mean', 'lin_speed_mean', 'rv_gyro_prox_mean',
       'lin_speed_prox_mean', 'rv_gyro__grv_mean'],
      dtype='object')

In [31]:
from sklearn import model_selection
results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

x_train, x_val, y_train, y_val = model_selection.train_test_split(
    x_train, y_train,
    test_size=0.2,
    stratify = y_train,   
    random_state=37
)

for model_name, model in tree_classifiers.items():

    start_time = time.time()
    model.fit(x_train, y_train)
    total_time = time.time() - start_time
    
    pred = model.predict(x_test)  
    
    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_test, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_test, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)


results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

/opt/anaconda3/envs/ml/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:11:19] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Learning rate set to 0.084978
0:	learn: 1.2649722	total: 98.6ms	remaining: 1m 38s
1:	learn: 1.1645129	total: 139ms	remaining: 1m 9s
2:	learn: 1.0847289	total: 167ms	remaining: 55.4s
3:	learn: 1.0153951	total: 325ms	remaining: 1m 20s
4:	learn: 0.9540361	total: 388ms	remaining: 1m 17s
5:	learn: 0.9019539	total: 438ms	remaining: 1m 12s
6:	learn: 0.8592600	total: 483ms	remaining: 1m 8s
7:	learn: 0.8210129	total: 546ms	remaining: 1m 7s
8:	learn: 0.7862344	total: 607ms	remaining: 1m 6s
9:	learn: 0.7562218	total: 637ms	remaining: 1m 3s
10:	learn: 0.7270252	total: 707ms	remaining: 1m 3s
11:	learn: 0.7033050	total: 807ms	remaining: 1m 6s
12:	learn: 0.6818381	total: 854ms	remaining: 1m 4s
13:	learn: 0.6602116	total: 926ms	remaining: 1m 5s
14:	lear

154:	learn: 0.1993787	total: 12.8s	remaining: 1m 9s
155:	learn: 0.1986189	total: 12.9s	remaining: 1m 9s
156:	learn: 0.1977198	total: 13s	remaining: 1m 9s
157:	learn: 0.1972684	total: 13.1s	remaining: 1m 10s
158:	learn: 0.1957656	total: 13.3s	remaining: 1m 10s
159:	learn: 0.1949571	total: 13.3s	remaining: 1m 10s
160:	learn: 0.1943087	total: 13.5s	remaining: 1m 10s
161:	learn: 0.1937720	total: 13.6s	remaining: 1m 10s
162:	learn: 0.1933335	total: 13.6s	remaining: 1m 10s
163:	learn: 0.1926586	total: 13.7s	remaining: 1m 9s
164:	learn: 0.1922564	total: 13.9s	remaining: 1m 10s
165:	learn: 0.1913144	total: 14s	remaining: 1m 10s
166:	learn: 0.1905810	total: 14.2s	remaining: 1m 10s
167:	learn: 0.1897185	total: 14.3s	remaining: 1m 10s
168:	learn: 0.1889927	total: 14.4s	remaining: 1m 10s
169:	learn: 0.1884594	total: 14.6s	remaining: 1m 11s
170:	learn: 0.1876227	total: 14.6s	remaining: 1m 10s
171:	learn: 0.1868791	total: 14.8s	remaining: 1m 11s
172:	learn: 0.1856225	total: 14.9s	remaining: 1m 11s
1

312:	learn: 0.1206286	total: 27.6s	remaining: 1m
313:	learn: 0.1203854	total: 27.7s	remaining: 1m
314:	learn: 0.1201205	total: 27.8s	remaining: 1m
315:	learn: 0.1200191	total: 27.9s	remaining: 1m
316:	learn: 0.1196513	total: 28s	remaining: 1m
317:	learn: 0.1193501	total: 28s	remaining: 1m
318:	learn: 0.1189620	total: 28.1s	remaining: 60s
319:	learn: 0.1187232	total: 28.1s	remaining: 59.8s
320:	learn: 0.1184260	total: 28.2s	remaining: 59.7s
321:	learn: 0.1181516	total: 28.3s	remaining: 59.6s
322:	learn: 0.1180420	total: 28.3s	remaining: 59.4s
323:	learn: 0.1178075	total: 28.4s	remaining: 59.2s
324:	learn: 0.1175692	total: 28.4s	remaining: 59.1s
325:	learn: 0.1170965	total: 28.5s	remaining: 59s
326:	learn: 0.1164399	total: 28.6s	remaining: 58.9s
327:	learn: 0.1161481	total: 28.7s	remaining: 58.9s
328:	learn: 0.1159727	total: 28.8s	remaining: 58.7s
329:	learn: 0.1157881	total: 28.8s	remaining: 58.5s
330:	learn: 0.1156978	total: 28.9s	remaining: 58.4s
331:	learn: 0.1155431	total: 29s	remai

475:	learn: 0.0846149	total: 39s	remaining: 42.9s
476:	learn: 0.0843163	total: 39.1s	remaining: 42.8s
477:	learn: 0.0840880	total: 39.1s	remaining: 42.7s
478:	learn: 0.0837725	total: 39.1s	remaining: 42.6s
479:	learn: 0.0836528	total: 39.2s	remaining: 42.5s
480:	learn: 0.0835289	total: 39.3s	remaining: 42.4s
481:	learn: 0.0832277	total: 39.3s	remaining: 42.3s
482:	learn: 0.0831088	total: 39.4s	remaining: 42.2s
483:	learn: 0.0828111	total: 39.5s	remaining: 42.1s
484:	learn: 0.0827155	total: 39.5s	remaining: 42s
485:	learn: 0.0824753	total: 39.6s	remaining: 41.9s
486:	learn: 0.0822292	total: 39.6s	remaining: 41.8s
487:	learn: 0.0820930	total: 39.7s	remaining: 41.6s
488:	learn: 0.0819879	total: 39.8s	remaining: 41.5s
489:	learn: 0.0818732	total: 39.8s	remaining: 41.5s
490:	learn: 0.0817660	total: 39.9s	remaining: 41.4s
491:	learn: 0.0815590	total: 39.9s	remaining: 41.2s
492:	learn: 0.0813221	total: 40s	remaining: 41.1s
493:	learn: 0.0811359	total: 40s	remaining: 41s
494:	learn: 0.0810637	

634:	learn: 0.0629865	total: 48.5s	remaining: 27.9s
635:	learn: 0.0628933	total: 48.6s	remaining: 27.8s
636:	learn: 0.0627760	total: 48.6s	remaining: 27.7s
637:	learn: 0.0626539	total: 48.7s	remaining: 27.6s
638:	learn: 0.0625661	total: 48.7s	remaining: 27.5s
639:	learn: 0.0623886	total: 48.8s	remaining: 27.5s
640:	learn: 0.0623317	total: 48.8s	remaining: 27.4s
641:	learn: 0.0622737	total: 48.9s	remaining: 27.3s
642:	learn: 0.0621392	total: 48.9s	remaining: 27.2s
643:	learn: 0.0619740	total: 49s	remaining: 27.1s
644:	learn: 0.0617982	total: 49s	remaining: 27s
645:	learn: 0.0617532	total: 49s	remaining: 26.9s
646:	learn: 0.0615917	total: 49.1s	remaining: 26.8s
647:	learn: 0.0614967	total: 49.1s	remaining: 26.7s
648:	learn: 0.0612669	total: 49.2s	remaining: 26.6s
649:	learn: 0.0611052	total: 49.3s	remaining: 26.5s
650:	learn: 0.0609564	total: 49.3s	remaining: 26.4s
651:	learn: 0.0607651	total: 49.4s	remaining: 26.3s
652:	learn: 0.0606448	total: 49.4s	remaining: 26.3s
653:	learn: 0.060558

795:	learn: 0.0489383	total: 58.4s	remaining: 15s
796:	learn: 0.0488152	total: 58.5s	remaining: 14.9s
797:	learn: 0.0487709	total: 58.6s	remaining: 14.8s
798:	learn: 0.0487392	total: 58.6s	remaining: 14.7s
799:	learn: 0.0486381	total: 58.7s	remaining: 14.7s
800:	learn: 0.0485225	total: 58.7s	remaining: 14.6s
801:	learn: 0.0484599	total: 58.8s	remaining: 14.5s
802:	learn: 0.0484019	total: 58.8s	remaining: 14.4s
803:	learn: 0.0483646	total: 58.8s	remaining: 14.3s
804:	learn: 0.0483375	total: 58.9s	remaining: 14.3s
805:	learn: 0.0482722	total: 59s	remaining: 14.2s
806:	learn: 0.0481828	total: 59.1s	remaining: 14.1s
807:	learn: 0.0480959	total: 59.2s	remaining: 14.1s
808:	learn: 0.0480414	total: 59.2s	remaining: 14s
809:	learn: 0.0479737	total: 59.3s	remaining: 13.9s
810:	learn: 0.0479119	total: 59.3s	remaining: 13.8s
811:	learn: 0.0478682	total: 59.4s	remaining: 13.8s
812:	learn: 0.0478001	total: 59.5s	remaining: 13.7s
813:	learn: 0.0477239	total: 59.5s	remaining: 13.6s
814:	learn: 0.0476

955:	learn: 0.0391636	total: 1m 8s	remaining: 3.13s
956:	learn: 0.0391415	total: 1m 8s	remaining: 3.06s
957:	learn: 0.0391267	total: 1m 8s	remaining: 2.99s
958:	learn: 0.0390590	total: 1m 8s	remaining: 2.91s
959:	learn: 0.0389781	total: 1m 8s	remaining: 2.84s
960:	learn: 0.0389394	total: 1m 8s	remaining: 2.77s
961:	learn: 0.0389003	total: 1m 8s	remaining: 2.7s
962:	learn: 0.0388147	total: 1m 8s	remaining: 2.63s
963:	learn: 0.0387644	total: 1m 8s	remaining: 2.56s
964:	learn: 0.0386919	total: 1m 8s	remaining: 2.48s
965:	learn: 0.0386504	total: 1m 8s	remaining: 2.41s
966:	learn: 0.0385797	total: 1m 8s	remaining: 2.34s
967:	learn: 0.0385180	total: 1m 8s	remaining: 2.27s
968:	learn: 0.0384979	total: 1m 8s	remaining: 2.2s
969:	learn: 0.0384496	total: 1m 8s	remaining: 2.13s
970:	learn: 0.0384179	total: 1m 8s	remaining: 2.06s
971:	learn: 0.0383951	total: 1m 8s	remaining: 1.98s
972:	learn: 0.0383378	total: 1m 8s	remaining: 1.91s
973:	learn: 0.0382722	total: 1m 8s	remaining: 1.84s
974:	learn: 0.

,Model,Accuracy,Bal Acc.,Time
1,AdaBoost,71.170084,74.785208,6.777115
2,Skl HistGBM,65.621230,63.768973,28.292895
3,Skl GBM,65.379976,62.949182,24.199113
4,XGBoost,64.897467,61.918456,20.894008
5,LightGBM,64.294331,62.893386,13.434816
6,Extra Trees,63.691194,67.764471,6.583256
7,CatBoost,61.399276,60.889342,77.993909
8,Random Forest,61.278649,64.973292,7.828248
9,Decision Tree,49.336550,53.919935,5.479943


In [33]:
from sklearn.metrics import classification_report,confusion_matrix

print(confusion_matrix(y_test,pred))
print(classification_report(y_test, pred))

[[ 50   2  23   1]
 [  0  20  24   0]
 [  5  26 150  17]
 [ 44  24 154 289]]
              precision    recall  f1-score   support

       Still       0.51      0.66      0.57        76
       Train       0.28      0.45      0.34        44
     Vehicle       0.43      0.76      0.55       198
     Walking       0.94      0.57      0.71       511

    accuracy                           0.61       829
   macro avg       0.54      0.61      0.54       829
weighted avg       0.74      0.61      0.64       829



In [45]:
len(y_test)

829